In [4]:
import datetime as dt
import numpy as np
import xarray as xr 

In [2]:
def power_spectra_plot(forecast_file,var,level,x,y,speedy_file=None):
    time_step = 1/14540
    time_slice = slice(1464,17000)
    time_slice_truth = slice(1460,17000)
    time_slice_speedy = slice(1432,17000)
    x = slice(x-5,x+5)
    y = slice(y-5,y+5)

    startdate = dt(2001,1,1,0)
    enddate = dt(2011,12,31,23)
    era_data, era_ps, temp_climo = get_truth_data(startdate,enddate,var,level,x,y)

    print('np.shape(era_data)',np.shape(era_data))
    era_data = lin_interp(era_data,era_ps,[950.0,1000.0])
    era_data = era_data[:,0,:,:]

    ds_forecast = xr.open_dataset(forecast_file)

    forecast_data = ds_forecast[var].sel(Timestep=time_slice,Sigma_Level=level,Lon=x,Lat=y)
    forecast_ps = ds_forecast['logp'].sel(Timestep=time_slice,Lon=x,Lat=y)
   
    forecast_data = lin_interp(forecast_data.values,forecast_ps.values,[950.0,1000.0])
    forecast_data = forecast_data[:,0,:,:]

    if speedy_file:
       ds_speedy =  xr.open_dataset(speedy_file)
       speedy_data = ds_speedy[var].sel(Timestep=time_slice_speedy,Sigma_Level=level,Lon=x,Lat=y)
       speedy_ps = ds_speedy['logp'].sel(Timestep=time_slice_speedy,Lon=x,Lat=y)
       speedy_data = lin_interp(speedy_data.values,speedy_ps.values,[950.0,1000.0])
       speedy_data = speedy_data[:,0,:,:]


    averaged_power_spectrum_era, averaged_power_spectrum_hybrid, averaged_power_spectrum_speedy, freq, idx = average_power_spectrum(era_data,forecast_data,speedy_data=speedy_data)

    start_index = dt(2009,12,27,0) - dt(2001,1,1,0)
    start_index = start_index.days * 4
    end_index = dt(2010,12,27,0) - dt(2001,1,1,0)#startdate
    end_index = end_index.days * 4
    plot_times_indices_hybrid = np.arange(start_index,end_index)
    print('end_index hybrid',end_index)
    print(np.shape(forecast_data))

    start_index = dt(2009,12,27,0) - dt(2001,1,1,0)
    start_index = start_index.days * 4
    end_index = dt(2010,12,27,0) - dt(2001,1,1,0)
    end_index = end_index.days * 4
    plot_times_indices_era = np.arange(start_index,end_index)
    
    xlabels_cal = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']
    xlabels_indices = [0,124,236,360,480,604,724,848,972,1092,1216,1336]

    plot_times = np.arange(0,1460)

    era_data_plot = np.average(era_data[plot_times_indices_era],axis=(1,2))
    forecast_data_plot = np.average(forecast_data[plot_times_indices_hybrid],axis=(1,2)) 
    speedy_data_plot = np.average(speedy_data[plot_times_indices_hybrid],axis=(1,2))

    era_data_mean = np.average(temp_climo[:,:,0,:,:]-273.15,axis=(0,2,3))
    era_data_sd = np.std(temp_climo[:,:,0,:,:]-273.15,axis=(0,2,3))


    upper_2sd_range = era_data_mean+era_data_sd*2
    lower_2sd_range = era_data_mean-era_data_sd*2 

    hybrid_out = outside_range(np.average(forecast_data,axis=(1,2)),dt(2001,1,1,0),dt(2010,12,31,23),upper_2sd_range,lower_2sd_range)
    speedy_out = outside_range(np.average(speedy_data,axis=(1,2)),dt(2001,1,1,0),dt(2010,12,31,23),upper_2sd_range,lower_2sd_range)

    print('Percent outside 2 SD hybrid',hybrid_out)
    print('Percent outside 2 SD SPEEDY',speedy_out)
    fig, axes = plt.subplots(3,1,figsize=(15,10),constrained_layout=True)

    ### 
    ax1 = axes[0]
    ax1.set_title('Power Spectrum',fontsize=18,fontweight="bold")
    ax1.loglog(freq[idx],averaged_power_spectrum_era[idx],label='ERA 5',color='#e41a1c')#,zorder=10)
    ax1.loglog(freq[idx],averaged_power_spectrum_hybrid[idx],label='Hybrid',color='#377eb8',zorder=10)

    ax1.legend(fontsize=18)
   
    print('np.max(freq[idx])',np.max(freq[idx]))
    print('daily power era',averaged_power_spectrum_era[np.where(freq == 1.0)])
    print('daily power hybrid',averaged_power_spectrum_hybrid[np.where(freq == 1.0)])
    print('diff power',averaged_power_spectrum_era[np.where(freq == 1.0)]-averaged_power_spectrum_hybrid[np.where(freq == 1.0)])
    ax1.set_xlim(1/(5*365.0),2.01) 
    ax1.set_ylim(np.min([np.min(averaged_power_spectrum_speedy[idx]),np.min(averaged_power_spectrum_era[idx])]),2*10**10)#np.max([np.max(averaged_power_spectrum_hybrid[idx]),np.max(averaged_power_spectrum_era[idx])]))
    ax1.minorticks_off()
    ax1.set_ylabel('$[Kelvin]^2$',fontsize=18)

    ax1.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off'  # labels along the bottom edge are off)
        )

    for tic in ax1.xaxis.get_major_ticks():
        tic.tick1On = tic.tick2On = False
        tic.label1On = tic.label2On = False
    ax1.grid() 


    ###
    ax2 = axes[1]
    ax2.sharex(ax1)
    ax2.sharex(ax1)
    #ax2.loglog(freq[idx],averaged_power_spectrum_hybrid[idx],label='Hybrid',color='#377eb8') 
    ax2.loglog(freq[idx],averaged_power_spectrum_era[idx],label='ERA 5',color='#e41a1c')#,zorder=10)
    if speedy_file:
       ax2.semilogy(freq[idx],averaged_power_spectrum_speedy[idx],label='SPEEDY',color='#4daf4a',zorder=10)

    ax2.legend(fontsize=18)

    xlabels_indices = [10**-3,1/365.0,10**-2,1/30.0,10**-1,1/7.0,1,2]
    xlabels_cal = [r'$10^{-3}$','Annual',r'$10^{-2}$','Monthly',r'$10^{-1}$','Weekly','Daily','12 hrs']
    ax2.set_xticks(xlabels_indices)
    ax2.set_xticklabels(xlabels_cal,fontsize=16,rotation = 45)
    ax2.set_xlim(1/(5*365.0),2.01)#np.min(freq[idx]),2.01)
    ax2.set_ylim(np.min([np.min(averaged_power_spectrum_speedy[idx]),np.min(averaged_power_spectrum_era[idx])]),2*10**10)
    ax2.grid()
    ax2.minorticks_off()
    ax2.set_ylabel('$[Kelvin]^2$',fontsize=18)
    ax2.set_xlabel('Frequency (1/day)',fontsize=18)

    plt.setp(ax1.get_xticklabels(), visible=False)


    xlabels_cal = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']
    xlabels_indices = [0,124,236,360,480,604,724,848,972,1092,1216,1336]

    ax = axes[2]
    ax.set_title('950 hPa Temperature',fontsize=18,fontweight="bold")
    ax.plot(plot_times,forecast_data_plot - 273.15,label='Hybrid',color='#377eb8')

    #sd_1 = ax.fill_between(plot_times,era_data_mean+era_data_sd,era_data_mean-era_data_sd, facecolor='grey', alpha=0.4)
    sd_2 = ax.fill_between(plot_times,era_data_mean+era_data_sd*2,era_data_mean-era_data_sd*2, facecolor='grey', alpha=0.4)

    if speedy_file:
       ax.plot(plot_times,speedy_data_plot - 273.15,label='SPEEDY',color='#4daf4a',linewidth=3)
    l1 = ax.legend(fontsize=14,loc=1)
    l2 = ax.legend([sd_2],['2 Sigma Range'],fontsize=14,loc=8)
    ax.add_artist(l1)
    ax.set_xticks(xlabels_indices)
    ax.set_yticks(np.arange(5,45,5))
    ax.set_yticklabels(np.arange(5,45,5),fontsize=14)
    ax.set_xticklabels(xlabels_cal,fontsize=16,rotation = 45)
    ax.set_ylabel('Celsius',fontsize=18)
    ax.set_xlim(0,1460)

In [3]:
def average_power_spectrum(era_data,forecast_data,speedy_data=None):
    time_step = 1/4.0#1/14540
    power_spectra_era = np.zeros((14540,np.shape(era_data)[1],np.shape(era_data)[2]))
    power_spectra_forecast = np.zeros((14540,np.shape(era_data)[1],np.shape(era_data)[2]))
    power_spectra_speedy = np.zeros((14540,np.shape(era_data)[1],np.shape(era_data)[2]))

    hamming = np.hanning(14540)
    for i in range(np.shape(era_data)[1]):
        for j in range(np.shape(era_data)[2]):

           fft_era = np.fft.fft(era_data[0:14540,i,j]*hamming)
           fft_forecast = np.fft.fft(forecast_data[0:14540,i,j]*hamming)

           power_spectra_era[:,i,j] = np.abs(fft_era)**2.0
           power_spectra_forecast[:,i,j]= np.abs(fft_forecast)**2.0

           if speedy_data is not None:
              fft_speedy = np.fft.fft(speedy_data[0:14540,i,j]*hamming)
              power_spectra_speedy[:,i,j] = np.abs(fft_speedy)**2.0

           print(np.shape(power_spectra_speedy),'power_spectra_speedy shape')
           freq = np.fft.fftfreq(len(speedy_data[0:14540,i,j]),time_step)

           idx = np.argsort(freq)
           idx = idx[int(len(idx)/2)::]

           #freq = 4*(freq/14540)

    averaged_power_spectrum_era = np.average(power_spectra_era,axis=(1,2))
    averaged_power_spectrum_hybrid = np.average(power_spectra_forecast,axis=(1,2))
    averaged_power_spectrum_speedy = np.average(power_spectra_speedy,axis=(1,2))

    return averaged_power_spectrum_era, averaged_power_spectrum_hybrid, averaged_power_spectrum_speedy, freq, idx

In [ ]:
forecast_file = 